In [9]:
a = [1,4,3,4]
import random

In [22]:
import json
attr_dict_file = 'dataset/vocab/word_dict.json'
with open(attr_dict_file, 'r') as f:
    attr_dict = json.load(f)

In [24]:
attr_dict['无拉链']

KeyError: '无拉链'

In [14]:
import torch
a = torch.tensor([i for i in range(10)])

In [1]:
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm 
import json 
import torch 

class TitleDataset(Dataset):
    def __init__(self, input_filename, is_train):
        self.is_train = is_train
        self.items = []
        for file in input_filename.split(','):
            with open(file, 'r') as f:
                for line in tqdm(f):
                    item = json.loads(line)
                    if self.is_train:
                        if item['match']['图文']: # 训练集图文必须匹配
                            self.items.append(item)
                    else:
                        self.items.append(item)
                
    def __len__(self):
        return len(self.items)
    
    def __getitem__(self, idx):
        image = torch.tensor(self.items[idx]['feature'])
        words = self.items[idx]['vocab_split']
        if self.is_train:
            return image, words
        else:
            label = self.items[idx]['match']['图文']
            return image, title, label

In [2]:
def collate_fn(batch):
    tensors = []
    splits = []
    for feature, split in batch:
        tensors.append(feature)
        splits.append(split)
    tensors = torch.stack(tensors)
    return tensors, splits
        

In [3]:
train_file = 'data/split_word/fine500_sample.txt'
train_dataset = TitleDataset(train_file, is_train=True)
train_dataloader = DataLoader(
        train_dataset,
        batch_size=3,
        shuffle=True,
        num_workers=8,
        pin_memory=True,
        collate_fn=collate_fn,
        drop_last=True
    )

502it [00:00, 3186.77it/s]


In [4]:
train_dataset[0]

(tensor([1.1288, 0.0095, 0.0517,  ..., 0.4841, 1.4008, 1.3691]),
 ['春季', '微喇裤', '牛仔裤', '蓝色', '常规', '厚度', '九分裤', '女装'])

In [5]:
for batch in train_dataloader:
    features, splits = batch
    print(features.shape)
    print(splits)
    break

torch.Size([3, 2048])
[['单肩包', '休闲', '运动包', '黑色'], ['休闲', '系带', '高帮', '男士', '帆布鞋', '秋季', '白', '黑', '卡通', '动漫'], ['翻领', '灰色', '常规', '厚度', '短款', '男装', '男士', '夹克', '标准型']]


In [6]:
loader = iter(train_dataloader)

In [7]:
loader.next()

[tensor([[ 0.9584,  0.0069,  0.0213,  ..., -1.3852,  0.9849, -0.0526],
         [-0.0550,  0.0022,  0.0114,  ..., -0.7029, -1.3373,  1.3214],
         [-0.2397, -0.0084,  0.0152,  ..., -0.3304,  0.3013,  1.2746]]),
 [['加厚', '连帽', '中长款', '大衣', '秋季', '蓝色', '拉链', '童装'],
  ['黑色', '秋季', '马甲', '标准型', '常规', '厚度', '开衫', '男装'],
  ['白色', '短袖', '衬衫', '秋季', '纯色', '童装']]]

In [9]:
import itertools
def get_negative_dict(file):
    with open(file, 'r') as f:
        new_dict = {}
        for attr, attrval_list in json.load(f).items():
            attr_dict = {}
            new_dict[attr] = attr_dict
            for x in attrval_list:
                l = attrval_list.copy()
                l.remove(x)
                x = x.split('=')
                l_noequal = list(map(lambda x: x.split('='), l))
                for k in x:
                    attr_dict[k] = list(itertools.chain.from_iterable(l_noequal))
    return new_dict

attr_dict_file = "data/original_data/attr_to_attrvals.json"
negative_dict = get_negative_dict(attr_dict_file)

In [70]:
import numpy as np
a = np.array([0.1,0.4,0.5])
np.random.choice([1,2,3], p=a)

3

In [14]:
from collections import OrderedDict
all_keys = list(ckpt.keys())
new_dict = OrderedDict()
for key in all_keys:
    if key.startswith('attr_encoder.'):
        new_dict[key[13:]] = ckpt[key]
    else:
        new_dict[key] = ckpt[key]

In [11]:
import os
import itertools
import torch 
import json
import numpy as np 
import random 
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm 

from model.attribute_matching import AttrModel


# 用来生成负样本的字典
def get_negative_dict(file):
    with open(file, 'r') as f:
        new_dict = {}
        for attr, attrval_list in json.load(f).items():
            attr_dict = {}
            new_dict[attr] = attr_dict
            for x in attrval_list:
                l = attrval_list.copy()
                l.remove(x)
                x = x.split('=')
                l_noequal = list(map(lambda x: x.split('='), l))
                for k in x:
                    attr_dict[k] = list(itertools.chain.from_iterable(l_noequal))
    return new_dict

attr_dict_file = "data/original_data/attr_to_attrvals.json"
negative_dict = get_negative_dict(attr_dict_file)

# 用来生成正样本的数据增强字典
def get_positive_dict(file):
    with open(file, 'r') as f:
        new_dict = {}
        for attr, attrval_list in json.load(f).items():
            attr_dict = {}
            new_dict[attr] = attr_dict
            for x in attrval_list:
                x = x.split('=')
                for k in x:
                    attr_dict[k] = x
    return new_dict
attr_dict_file = "data/original_data/attr_to_attrvals.json"
positive_dict = get_positive_dict(attr_dict_file)

In [4]:
import os
import re
import json
import torch
import numpy as np
import itertools
import tqdm

def load_attr_dict(file):
    # 读取属性字典
    with open(file, 'r') as f:
        attr_dict = {}
        for attr, attrval_list in json.load(f).items():
            attrval_list = list(map(lambda x: x.split('='), attrval_list))
            attr_dict[attr] = list(itertools.chain.from_iterable(attrval_list))
    return attr_dict

def match_attrval(title, attr, attr_dict):
    # 在title中匹配属性值
    attrvals = "|".join(attr_dict[attr])
    ret = re.findall(attrvals, title)
    # return "{}{}".format(attr, ''.join(ret))
    return "{}".format(''.join(ret))  

attr_dict_file = "data/original_data/attr_to_attrvals.json"
attr_dict = load_attr_dict(attr_dict_file)


In [5]:
title = '松紧带抓鸡松紧'
match_attrval(title, '裤门襟', attr_dict)

'松紧松紧'

In [16]:
title

'松紧带抓鸡松紧'

In [15]:
title.replace('松紧', '拉链')

'拉链带抓鸡拉链'

In [1]:
from ltp import LTP
from tqdm import tqdm
import json

In [2]:
ltp = LTP(path='base') 

In [3]:
# 取得属性字典并添加到词表
def get_dict(file):
    with open(file, 'r') as f:
        all_attr = []
        for attr, attrval_list in json.load(f).items():
            for x in attrval_list:
                x = x.split('=')
                for k in x:
                    all_attr.append(k)
    return all_attr
attr_dict_file = "data/original_data/attr_to_attrvals.json"
all_attr = get_dict(attr_dict_file)
extra_words = []
extra_words.append(['牛津布', '仿皮', '吸湿', '吸汗', '防滑', '抗冲击', '微弹', '加绒'])
extra_words.append(['上青', '上青色', '上青绿', '羊绒衫'])
extra_words.append(['休闲鞋', '工装鞋', '男包', '女包', '运动裤', '休闲裤', '加厚领'])
extra_words.append(['加厚', '薄款', '厚款', '短款', '短外套'])
extra_words.append(['不加绒', '无扣', '无弹力', '无弹', '无拉链'])
extra_words.append(['一粒扣', '两粒扣', '暗扣', '三粒扣', '系扣'])
extra_words.append(['大红色', '大花'])
for extra in extra_words:   
    all_attr = all_attr + extra

ltp.init_dict(path="user_dict.txt", max_window=6)
ltp.add_words(words=all_attr, max_window=6)

In [9]:
# 统计训练集的词表和词频
fine_file = 'data/train/fine45000.txt'
word_dict = {}
with open(fine_file, 'r') as f:
    for line in tqdm(f):
        item = json.loads(line)
        segment, _ = ltp.seg([item['title']])
        for word in segment[0]:
            word = word.upper() # 字母统一为大写
            if word in word_dict:
                word_dict[word] += 1
            else:
                word_dict[word] = 1

coarse_file = 'data/train/coarse89588.txt'
with open(coarse_file, 'r') as f:
    for line in tqdm(f):
        item = json.loads(line)
        segment, _ = ltp.seg([item['title']])
        for word in segment[0]:
            word = word.upper() # 字母统一为大写
            if word in word_dict:
                word_dict[word] += 1
            else:
                word_dict[word] = 1

45000it [08:45, 85.65it/s]
89588it [17:21, 85.99it/s]


In [10]:
# 为了不丢失花大量时间生成的词表先做一个拷贝
import copy 
copy_dict = copy.deepcopy(word_dict)

In [35]:
# 备用代码
# word_dict = copy.deepcopy(copy_dict)

In [37]:
# 统一大量颜色的别称
color_list = ['兰','蓝','灰','绿','粉','红','黄','青','紫','白','黑','骆','橙','杏','咖','棕','啡','褐','银','金','橘','藏']
keys = []
for key in  word_dict.keys():
    keys.append(key)
for key in keys:
    for i in key:
        if i in color_list and word_dict[key] < 50:
            if i in word_dict:
                word_dict[i] += word_dict[key]
            else:
                word_dict[i] = word_dict[key]


In [38]:
# 删除出现次数少的词
keys = []
for key in word_dict.keys():
    keys.append(key)
for key in keys:
    if word_dict[key] < 50:
        del word_dict[key]
    if key == '/':
        del word_dict[key]

In [15]:
# 保存词表
with open('word_dict.json', 'w') as f:
    json.dump(word_dict, f, ensure_ascii=False)

In [40]:
# 保存处理后的词表
with open('processed_word_dict.json', 'w') as f:
    json.dump(word_dict, f, ensure_ascii=False)

In [5]:
# 读入保存的词表
with open('processed_word_dict.json', 'r') as f:
    processed_word_dict = json.load(f)

In [6]:
processed_word_dict

{'春季': 11663,
 '微喇裤': 307,
 '牛仔裤': 4171,
 '蓝色': 6279,
 '常规': 26114,
 '厚度': 24087,
 '九分裤': 3254,
 '女装': 47717,
 '浅咖': 70,
 '修身型': 20147,
 '套头': 15191,
 '针织衫': 6258,
 '宽松型': 19726,
 '牛仔蓝': 189,
 '半高领': 3616,
 '长袖': 34415,
 '外套': 6731,
 '纯色': 25986,
 '秋季': 32528,
 '加厚': 16141,
 '灰色': 7153,
 '高领': 3727,
 '童装': 17785,
 '衬衫': 4284,
 '常规款': 18123,
 '休闲': 9649,
 '百搭': 5344,
 '男士': 14405,
 '棉服': 6540,
 '拉链': 14748,
 '中长款': 13700,
 '翻领': 5963,
 '男装': 37285,
 '微弹': 4159,
 '黑色': 32664,
 '冬季': 23523,
 '系带': 14200,
 '字母': 1299,
 '加绒裤': 3948,
 '墨绿': 243,
 '夏季': 8847,
 '标准型': 15138,
 'POLO领': 2249,
 'T恤': 6952,
 '短袖': 5876,
 '薄款': 6046,
 '白色': 8428,
 '正装裤': 1441,
 '长裤': 14112,
 '小脚裤': 1134,
 '立领': 4878,
 '长款': 1986,
 '女士': 13116,
 '风衣': 3717,
 '单排扣': 4040,
 '卡其色': 2447,
 '连帽': 12974,
 '印花': 1274,
 '儿童': 5896,
 '酒红色': 931,
 '仿皮': 3128,
 '皮衣': 7039,
 '娃娃领': 576,
 '暗扣': 168,
 '羊毛衫': 2715,
 'V领': 3525,
 '运动': 5232,
 '帆布鞋': 3086,
 '低帮': 8997,
 '轻便': 458,
 '卫衣': 7387,
 '圆领': 12853,
 '真皮': 2684,
 '透气': 2190,

In [4]:
# 生成分好的title词集
fine_file = 'data/train/fine45000.txt'
save_fine_file = 'data/train/fine45000_OnlySplit.txt'
rets = []
i = 0
with open(fine_file, 'r') as f:
    for line in tqdm(f):
        new_item = {}
        item = json.loads(line)
        segment, _ = ltp.seg([item['title']])
        for word in segment[0]:
            word = word.upper() # 字母统一为大写
        new_item['title_split'] = segment[0]
        # 更改保存的顺序，便于查看
        rets.append(json.dumps(new_item, ensure_ascii=False)+'\n')
        
        # if i>20000:
        #     break
        # i += 1
        
with open(save_fine_file, 'w') as f:
    f.writelines(rets)

45000it [08:51, 84.60it/s]


In [50]:
# 生成分好的title词集
fine_file = 'data/train/fine45000.txt'
save_fine_file = 'data/train/fine45000_Sample.txt'
rets = []
i = 0
with open(fine_file, 'r') as f:
    for line in tqdm(f):
        item = json.loads(line)
        segment, _ = ltp.seg([item['title']])
        for word in segment[0]:
            word = word.upper() # 字母统一为大写
        item['title_split'] = segment[0]
        # 更改保存的顺序，便于查看
        feature = item['feature']
        del item['feature']
        item['feature'] = feature
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
        if i>500:
            break
        i += 1
        
with open(save_fine_file, 'w') as f:
    f.writelines(rets)

501it [00:06, 80.09it/s]


In [48]:
coarse_file = 'data/train/coarse89588.txt'
save_coarse_file = 'data/train/coarse89588_S.txt'
rets = []
i = 0
with open(coarse_file, 'r') as f:
    for line in tqdm(f):
        item = json.loads(line)
        segment, _ = ltp.seg([item['title']])
        for word in segment[0]:
            word = word.upper() # 字母统一为大写
        item['title_split'] = segment[0]
        # 更改保存的顺序，便于查看
        feature = item['feature']
        del item['feature']
        item['feature'] = feature
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
        # if i>500:
        #     break
        # i += 1

with open(save_coarse_file, 'w') as f:
    f.writelines(rets)

89588it [19:08, 78.03it/s]
